In [1]:
import numpy as np
import pandas as pd
import random as rd 

`define problem` :<br>
we want to solve bounded fractional knapsack problem using __genetic algorithm__ which is ridiculous because this problem has **greedy solution** <br> <br> we define our gen as weight of i'st item in the list and it is some fractional number between *0* and *1*. As we know our chromosome as a list of gens so we define our chro class as below:

In [2]:
class Chro:
    def __init__(self , gen_list):
        self.gen_list = gen_list.copy()
    def size (self): 
        return len(self.gen_list)

`choose_and_remove`: this function get an iterable element and randomly choose an item from that and then remove that element from the input

In [3]:
def choose_and_remove(a):
    res = rd.choice(a)
    a.remove(res)
    return res

## Genetic class
### we want to maximize value subject to bounded weghit and bounded diversity 
##### init funcion :
- it gets all of the constant that need for execution
##### calculate_value : 
- actualy this is fitness function and goal is to maximize this function
##### single_chro  : 
- it returns a valide chromosome i will explain how it works in presentaion
##### mutate : 
- it get a chromosome and randomly chose one of gens then mutate that subject to weight and diversity are kept valid
##### do_mutate : 
- it randomly select some gens than mutate them
##### cross_ove : 
- this fucntion will  cross over two chromosome
##### do_cross_over :
- it get cross over pool and randomly cross over them
#####  get_carry : 
- it will send bests of the population to next generation directly
##### get_crossover_pool 
- it will return a pool that is needed by do_cross_over function
##### get_population
- generate initial population

In [4]:
class Genetic:
    def __init__(self , csv2 ,min_dif , max_dif , max_weight,min_val,population_size,p_crossover,p_mutate,carry_pool_size,density_list):
        self.snacks = csv2
        self.min_dif = min_dif
        self.max_dif = max_dif
        self.max_weight = max_weight
        self.min_val = min_val
        self.population_size= population_size
        self.p_mutate = p_mutate
        self.carry_pool_size = carry_pool_size
        self.density_list = density_list 
        
    def calculate_weight(self , chro):
        return sum(chro)

    def calculate_value(self , chro):
        res = 0
        for i in range(len(chro)): 
            total_weight = self.snacks["Available Weight"].iloc[i]
            total_value = self.snacks["Value"].iloc[i]
            res = res + ((chro[i]/total_weight)* total_value)
        return res

    def calculate_diversity(self , chro):
        res = 0
        for i in chro : 
            if (i == 0):
                res = res +1 
        return len(chro) - res

    def single_chro(self):        
        diversity = rd.randint(self.min_dif, self.max_dif)
        chro = [0 for x in range(len(self.snacks))]
        available_locations = list(range(len(self.snacks)))
        available_weight = self.max_weight
        for i in range(diversity):
            location = choose_and_remove(available_locations)
            chro[location] = rd.uniform(0 , min (self.snacks["Available Weight"].iloc[location], available_weight))
            available_weight -= chro[location]
        return chro  
    
    def is_valid_diversity(self , chro):
        if ((self.calculate_diversity(chro) < self.max_dif and self.calculate_diversity(chro)> self.min_dif)):
            return True
        else:
            return False
    def mutate(self,chro):
        diversity = self.calculate_diversity(chro)
        index = rd.randint(0, len(chro)-1)
        if(diversity == self.max_dif):
            while(chro[index]==0):
                index = rd.randint(0, len(chro)-1)
        w = self.calculate_weight(chro)
        chro[index] = rd.uniform(0 , min (self.snacks["Available Weight"].iloc[index], self.max_weight-w))
        return chro
    
    def do_mutate(self , mutate_pool):
        for i in range(len(mutate_pool)):
            l = [0 , 1]
            p = [1-self.p_mutate, self.p_mutate]
            f = np.random.choice(l,1,p=p)
            if (f == 1):
                mutate_pool[i] = self.mutate(mutate_pool[i])
        mutate_pool= sorted(mutate_pool,key = lambda x:self.calculate_value(x))
        self.population = mutate_pool
        return mutate_pool

    def cross_over(self , chro1 , chro2 ):
        valid_points = list(range (len(chro1)))
        while(len(valid_points)!=0):
            cros_point = choose_and_remove(valid_points)
            res1 = [0 for j in range(len(chro1))]
            res2 = [0 for j in range(len(chro1))]
            for i in range(len(chro1)):
                if (i < cros_point):
                    res1[i] = chro1[i]
                    res2[i] = chro2[i]
                else :
                    res1[i] = chro2[i]
                    res2[i] = chro1[i]
            
            if (not self. is_valid_diversity(res1) or not self.is_valid_diversity(res2)):
                continue
            else : 
                w1 = self.calculate_weight(res1)
                if ( w1> self.max_weight):
                    more = w1 - self . max_weight
                    index= 0
                    while (more > 0):
                        a = min (res1[self.density_list[index][0]] , more)
                        res1[self.density_list[index][0]] -= a
                        more -= a
                        index += 1 
                        pass
                w2 = self.calculate_weight(res2)
                if (w2> self.max_weight):
                    more = w2 - self . max_weight
                    index= 0
                    while (more > 0):
                        a =  min (res2[self.density_list[index][0]] , more)
                        res2[self.density_list[index][0]] -= a
                        more -= a
                        index += 1 
                        pass
                return res1 , res2
        return chro1,chro2
    def do_cross_over(self, pop_pool):
        res = [x for x in self.carry_pool]
        while(len(pop_pool)):
            chro1 = choose_and_remove(pop_pool)
            chro2 = choose_and_remove(pop_pool)
            ch1,ch2 = self.cross_over(chro1.gen_list,chro2.gen_list)
            res.append(ch1)
            res.append(ch2)
        res = sorted(res ,key= lambda x : self.calculate_value(x))
        self.population = res
        return res


    def get_carry(self):
        self.carry_pool = [None for i in range(self.carry_pool_size)]
        for i in range(self.carry_pool_size):
            self.carry_pool[i]=self.population[-1-i]       
        return self.carry_pool

    def get_crossover_pool(self):
        p = [self.calculate_value(x) for x in self.population]
        p = [x/sum(p) for x in p]
        po = [Chro(x) for x in self.population]
        a = np.random.choice(po,self.population_size - self.carry_pool_size ,p=p,replace=True)
        return list(a)
    def get_population(self):
        self.population = [[] for j in range(self.population_size)]
        for i in range(self.population_size):
            self.population[i] = self.single_chro()
        self.population=sorted(self.population , key = lambda x : self.calculate_value(x))
        return self.population

### and lets start algorithm

In [5]:
csv = pd.read_csv("snacks.csv")
density_list = [0 for i in range(len(csv))]
for i in range(len(csv)):
     density_list[i] = csv['Value'].iloc[i]/csv['Available Weight'].iloc[i]

density_list = list(enumerate(density_list))
density_list = sorted(density_list , key = lambda x : x[1])

In [11]:
g = Genetic(csv,0,19,20,35,200,0.9,0.6,10,density_list)
p = g.get_population()
for i in range(100):
    po_pol = g.get_crossover_pool()
    c = g.get_carry()
    e = g.do_cross_over(po_pol)
    e = g.do_mutate(e)
    if (g.calculate_value(e[-1])>= g.min_val):
        break
    print(g.calculate_value(e[-1]))
    print("weight: {}".format(g.calculate_weight(e[-1])))
print(e[-1])
print(g.calculate_value(e[-1]))


30.540263307632
weight: 19.99998655677856
30.79567464714281
weight: 19.649359248222716
34.538302430505
weight: 20.0
34.538302430505
weight: 20.0
34.538302430505
weight: 20.0
34.628699057753394
weight: 20.0
34.665433229192544
weight: 19.99401204426782
[0, 0.0, 0.0, 6.447512818629365, 0, 0, 0.0, 0.0, 4.167787450946035, 0, 0.0, 0, 0.0, 2.676137109105824, 0.0, 0.0, 6.708562621318778, 0.0, 0.0]
35.20112964050281


# questions: 
‍` 1:` <br> اگر بسیار کم باشد احتمال به جواب خوب رسیدن کم می شود و اگر زیاد باشد سرعت اجرای الگوریتم کم میشود
<br> ` 2: ` <br>
اگر در هر مرحله اندازه جمعیت زیاد شود سرعت الگوریتم کم تر شده ولی در جمعبت هایی با تعداد بیشتر احتمال رسیدن به جواب بیشتر می شود اگر حمعیت ها به سمت بهتر شدن جهش کنند
 <br> ` 3: ` <br> 
میوتیشن موجب می شود که الگوریتم در نقاط اوپتیمال محلی گیر نکند همچنین شاید کروموزومی خوب باشد و تنها یکی از ژن های آن موجب بهینه نبودن آن باشد بنابرین تغییرات اندک در کروموزوم ها احتمالا موجب پرش زیادی در ماکسیمم و میانگین جمعیت در هر نسل می شود
<br> ` 4: ` <br>
کم کردن اندازه جمعیت 
درصد میوتیشنی که گذاشتم نزدیکی های ۵۰ تا ۶۰ درصد 
استفاده از کری پول
<br> ` 5: ` <br>
بعله ممکن است از هیج نقطع ای قابل کراس اور شدن نباشند و هیچ وقت وارد جمعیتی که قرار است میوتیت شوند نشوند با این حال چون این ژن ها وزن زیادی دارند یعنی به نسبت ارزش زیادی هم دارند بنابرین مشکلی در کل ایجاد نخواهد شد و ژن های خوب بهتری بعد از چند نسل تولید خواهند شد. 
<br> ` 6: ` <br>
دقیقا همان کاری ک در کد خودم کردم مستقل از وجود یا عدم وجود جواب بهترین جواب را بعد از تعداد محدودی نسل چاپ خواهم کرد

